In [1]:
from datasets import load_dataset

dataset = load_dataset("dim/semeval_subtask2_conversations")
dataset

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 100%|██████████| 110/110 [00:00<00:00, 39389.86 examples/s]


DatasetDict({
    train: Dataset({
        features: ['conversation_ID', 'conversation', 'emotion-cause_pairs'],
        num_rows: 1264
    })
    test: Dataset({
        features: ['conversation_ID', 'conversation', 'emotion-cause_pairs'],
        num_rows: 110
    })
})

In [8]:
dataset["train"][9]

{'conversation_ID': 1159,
 'conversation': [{'emotion': 'neutral',
   'speaker': 'Ross',
   'text': 'Hi .',
   'utterance_ID': 1,
   'video_name': 'dia1159utt1.mp4'},
  {'emotion': 'surprise',
   'speaker': 'Chandler',
   'text': 'Hey . Soaps ? Shampoos ? Are you really taking all this stuff ?',
   'utterance_ID': 2,
   'video_name': 'dia1159utt2.mp4'},
  {'emotion': 'neutral',
   'speaker': 'Ross',
   'text': 'Why not ? It is built into the price of the room .',
   'utterance_ID': 3,
   'video_name': 'dia1159utt3.mp4'},
  {'emotion': 'surprise',
   'speaker': 'Chandler',
   'text': 'Yeah but you do not need ... What is this ?',
   'utterance_ID': 4,
   'video_name': 'dia1159utt4.mp4'},
  {'emotion': 'neutral',
   'speaker': 'Ross',
   'text': 'Thread !',
   'utterance_ID': 5,
   'video_name': 'dia1159utt5.mp4'},
  {'emotion': 'joy',
   'speaker': 'Chandler',
   'text': 'Score ! Where are the disposable cameras ?',
   'utterance_ID': 6,
   'video_name': 'dia1159utt6.mp4'},
  {'emotion'

In [12]:
(1, 2) in set([(1, 2)])

True

In [20]:
# new_dataset = []

# for i, conversation in enumerate(dataset["train"]):
#     cause_pairs = conversation["emotion-cause_pairs"]
#     future_cause = False
#     for cause_pair in cause_pairs:
#         initial_emotion = int(cause_pair[0].split("_")[0]) - 1
#         cause_emotion = int(cause_pair[1]) - 1
#         if cause_emotion > initial_emotion:
#             future_cause = True
#             print(initial_emotion, cause_emotion)

#     if future_cause:
#         print(i, cause_pairs)

#         print("==" * 10)
#         print("==" * 10)

In [26]:
import random

random.seed(42)
def generate_dataset(dataset):
    new_dataset = []
    for i, conversation in enumerate(dataset):
        cause_pairs = conversation["emotion-cause_pairs"]
        # print(i, cause_pairs)
        positive_pairs = []
        for cause_pair in cause_pairs:
            positive_example = {"initial": {}, "cause": {}, "label": 1}
            initial_emotion_pos = int(cause_pair[0].split("_")[0]) - 1
            initial_emotion = conversation["conversation"][initial_emotion_pos]
            cause_emotion_pos = int(cause_pair[1]) - 1
            cause_emotion = conversation["conversation"][cause_emotion_pos]
            positive_example["initial"] = initial_emotion
            positive_example["cause"] = cause_emotion

            positive_pairs.append((initial_emotion_pos, cause_emotion_pos))
            new_dataset.append(positive_example)

        new_dataset.extend(positive_pairs)
        positive_pairs = set(positive_pairs)
        negative_pairs = []
        for pos_i in range(len(conversation["conversation"])):
            for pos_j in range(len(conversation["conversation"])):
                pair = (pos_i, pos_j)
                if not pair in positive_pairs:
                    negative_pairs.append(pair)
        # print(len(negative_pairs), positive_pairs)
        
        if len(negative_pairs) > len(positive_pairs):
            negative_pairs = random.sample(negative_pairs, len(positive_pairs))
        # print("==" * 10)
        # print("==" * 10)
        new_dataset.extend(negative_pairs)
    
    return new_dataset

train_dataset = generate_dataset(dataset=dataset['train'])
test_dataset = generate_dataset(dataset=dataset['test'])

len(train_dataset), len(test_dataset)

(26821, 2542)

In [27]:
train_dataset[0]

{'initial': {'emotion': 'sadness',
  'speaker': 'Monica',
  'text': 'Mr . Heckles .',
  'utterance_ID': 1,
  'video_name': 'dia187utt1.mp4'},
 'cause': {'emotion': 'sadness',
  'speaker': 'Monica',
  'text': 'Mr . Heckles .',
  'utterance_ID': 1,
  'video_name': 'dia187utt1.mp4'},
 'label': 1}